In [1]:
from twikit import Client
import re
import string
import os
import json
from datetime import datetime
import time


printable = set(string.printable)


def sanitize(text):
    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'@\w+ ', ' ', text)

    text = " ".join(re.findall(r'\w+|[^\w\s]', text))
    
    text = ''.join(filter(lambda x: x in printable, text))
    text = text.lower().strip().rstrip()
    return text

In [2]:
client = Client('en-US')

clientDict = {'JoeNuts836498': 'techno34',
             'dylan_berndt': 'techno34'}

In [3]:
def retrieveTweets(userName, password, seenIds):
    client.login(auth_info_1=userName, password=clientDict[userName])
    client.save_cookies('cookies.json')
    client.load_cookies(path='cookies.json')

    try:
        return client.get_timeline(count=400, seen_tweet_ids=seenIds)
    except:
        return None

seenIds = []
path = 'data/'
files = os.listdir(path)
for fileName in files:
    if not fileName.endswith('.json'):
        continue
    file = open(path + fileName)
    fileData = json.load(file)
    for key, tweet in fileData.items():
        if key not in seenIds:
            seenIds.append(key)

import json
from datetime import datetime
import time

while True:
    data = {}
    
    for userName in clientDict:
        client.login(auth_info_1=userName, password=clientDict[userName])
        client.save_cookies('cookies.json')
        client.load_cookies(path='cookies.json')

        try:
            tweets = client.get_timeline(count=400, seen_tweet_ids=seenIds)
        except:
            continue
    
        usefulCount = 0
        replyCount = 0
        
        try:
            for tweet in tweets:
                if not tweet:
                    continue
                if tweet.id in seenIds:
                    continue
                seenIds.append(tweet.id)
                t = client.get_tweet_by_id(tweet.id)
                if not t.replies:
                    continue
                if len(t.replies) > 0:
                    usefulCount += 1
                    replyCount += len(t.replies)
                    sanitized = [sanitize(reply.text) for reply in t.replies]
                    replies = [reply for reply in sanitized if len(reply) > 0]
                    tweetData = {"text": sanitize(t.text), "replies": replies}
                    data[t.id] = tweetData
                    print(userName, "Tweets:", usefulCount, "Replies:", replyCount, end='\r')
        except Exception as e:
            print(e)
            continue
        print()
    
    if len(data) < 1:
        quit()
    
    now = datetime.now()
    timeName = now.strftime("%y.%m.%d.%H.%M.%S")
    
    file = open("data/timeline " + timeName + ".json", 'w+')
    json.dump(data, file, indent=4)
    file.close()

    time.sleep(600)

In [4]:
dataFile = open('twitterDialog/seenIds.txt')
read = dataFile.readlines()
if len(read) > 0:
    seen = read[0].split()
else:
    seen = []

In [5]:
def recursiveTweet(tweetId):
    current = client.get_tweet_by_id(tweetId)

    compiled = [sanitize(current.text) + " __eou__ "]
    ids = [current.id]

    if len(current.replies) < 1:
        return compiled, ids

    for reply in current.replies:
        threadData, foundIds = recursiveTweet(reply.id)

        threadCompiled = [sanitize(current.text) + " __eou__ " + thread for thread in threadData]
        compiled.extend(threadCompiled)

        ids.extend(foundIds)

    return compiled, ids

In [6]:
def saveThreads(threadData, newIds):
    if len(threadData) < 1:
        return

    seenFile = open("twitterDialog/seenIds.txt", "a+")
    seenFile.write(" ".join(newIds))
    seenFile.close()

    now = datetime.now()
    timeName = now.strftime("%y.%m.%d.%H.%M.%S")
    threadFile = open("twitterDialog/timeline " + timeName + ".txt", 'w+')
    text = "\n".join(threadData)
    threadFile.write(text)
    threadFile.close()

In [ ]:
while True:
    data = []
    checkedIds = []

    for user in clientDict:
        tweets = retrieveTweets(user, clientDict[user], seen)

        usefulCount = 0
        replyCount = 0

        try:
            for tweet in tweets:
                if not tweet:
                    continue
                if tweet.id in seen:
                    continue
                seen.append(tweet.id)

                retrieved, threadIds = recursiveTweet(tweet.id)
                checkedIds.extend(threadIds)

                data.extend(retrieved)
                usefulCount += 1
                replyCount += len(retrieved)

                print(user, "Tweets:", usefulCount, "Threads:", replyCount, end='\r')

        except Exception as e:
            print("\nSaving...")
            saveThreads(data, checkedIds)
            continue

        print()

    print("Saving...")
    saveThreads(data, checkedIds)
    time.sleep(200)



Saving...
